In [1]:
# pull in dependencies needed from sportsreference api
from sportsreference.mlb.teams import Teams
from sportsreference.mlb.teams import Roster
from sportsreference.mlb.roster import Player

# pull in all other dependencies needed
import itertools
import pandas as pd
import time

In [2]:
# to create a list of player ids:
# set up an empty array that we will append to
player_list = []

# 1) create a loop to go through years
for year in range(1876,2020):

# 2) create a loop to go through teams and pull out the player names and their ids
    for team in Teams(year):
        player_data = Roster(team.abbreviation, year = year, slim=True).players
        player_list.append(player_data)

In [3]:
# convert the dictionay of arrays to an array of arrays using list comprehension
info = [list(x.keys()) for x in player_list]

In [4]:
# flatten the list using itertools
merged = list(itertools.chain(*info))

In [5]:
# create a pandas data frame that will show all of the player ids
df = pd.DataFrame(merged)

In [6]:
# use .unique() on the df to clear out any duplicate players
unique_player_id = df[0].unique()

In [7]:
# to create a list of data frames for each player's stats:
# set up an empty array that we will append to
player_df_list = []

# create a loop that will find each players' id and create a data frame of relevant statistics
for index, player_id in enumerate(unique_player_id):
    try:
        player_df = Player(player_id).dataframe[["player_id", "name", "at_bats", "runs", "hits", "runs_batted_in", "bases_on_balls", "batting_average", "doubles", "triples", "home_runs", "stolen_bases"]]
        player_df_list.append(player_df)
        print(index, len(unique_player_id))
    except:
        print("skip")
    time.sleep(1)

0 19494
1 19494
2 19494
3 19494
4 19494
5 19494
6 19494
7 19494
8 19494
9 19494
10 19494
11 19494
12 19494
13 19494
14 19494
15 19494
16 19494
17 19494
18 19494
19 19494
20 19494
21 19494
22 19494
23 19494
skip
25 19494
26 19494
27 19494
28 19494
29 19494
skip
31 19494
32 19494
33 19494
34 19494
35 19494
36 19494
37 19494
38 19494
39 19494
40 19494
41 19494
42 19494
43 19494
44 19494
45 19494
46 19494
47 19494
48 19494
49 19494
50 19494
51 19494
52 19494
53 19494
54 19494
55 19494
56 19494
57 19494
58 19494
59 19494
60 19494
61 19494
skip
63 19494
64 19494
skip
66 19494
67 19494
68 19494
69 19494
70 19494
71 19494
skip
73 19494
74 19494
skip
skip
skip
78 19494
79 19494
skip
81 19494
82 19494
skip
84 19494
85 19494
skip
87 19494
skip
89 19494
90 19494
91 19494
92 19494
skip
94 19494
95 19494
skip
skip
98 19494
skip
100 19494
skip
skip
skip
104 19494
105 19494
106 19494
107 19494
108 19494
109 19494
110 19494
111 19494
112 19494
113 19494
114 19494
115 19494
116 19494
117 19494
118 19494

In [8]:
# Use .loc function in pandas to just obtain the career stats for every player
career_df_list = []
for df in player_df_list:
    career_df_list.append(df.loc["Career"])

In [9]:
# use concat to re-write the list of data frames as a single data frame
clean_df = pd.concat(career_df_list)

# and drop the duplicates of names using either 'player_id' or 'name'
mlb_history_df = clean_df.drop_duplicates(subset=['player_id'])
mlb_history_df.head()

,player_id,name,at_bats,runs,hits,runs_batted_in,bases_on_balls,batting_average,doubles,triples,home_runs,stolen_bases
Career,whitede01,Deacon White,6624,1140.0,2067.0,988,308.0,0.312,270,98,24.0,70
Career,mcveyca01,Cal McVey,2513,555.0,869.0,449,30.0,0.346,133,44,11.0,26
Career,barnero01,Ross Barnes,2391,698.0,860.0,346,116.0,0.36,146,47,6.0,103
Career,peterjo01,John Peters,2693,373.0,748.0,249,25.0,0.278,92,12,3.0,14
Career,ansonca01,Cap Anson,10281,1999.0,3435.0,2075,984.0,0.334,582,142,97.0,277


In [10]:
# read in the hall of fame csv
hall_of_fame_df = pd.read_csv('Resources/Hall_of_Fame_List.csv')
hall_of_fame_df.head()

,Year,Name,Primary position,Primary team,Career,Induction method,Vote %
0,1936.0,Ty Cobb,CF,Detroit Tigers,1905–1928,BBWAA,98.23%
1,1936.0,Walter Johnson,P,Washington Senators,1907–1927,BBWAA,83.63%
2,1936.0,Christy Mathewson,P,New York Giants,1900–1916,BBWAA,90.71%
3,1936.0,Babe Ruth,RF,New York Yankees,1914–1935,BBWAA,95.13%
4,1936.0,Honus Wagner,SS,Pittsburgh Pirates,1897–1917,BBWAA,95.13%


In [11]:
# clean the csv file to show only the names of the hall of famers and status
hall_of_fame_df = hall_of_fame_df.dropna()
hof_df = hall_of_fame_df.rename(columns={"Name":"name", "Primary position": "position"})
hof_cleaned = hof_df[["name", "position"]]
hof_cleaned.head()

,name,position
0,Ty Cobb,CF
1,Walter Johnson,P
2,Christy Mathewson,P
3,Babe Ruth,RF
4,Honus Wagner,SS


In [12]:
# join the no_duplicates_df with hof_cleaned to show career stats and hall of fame status
merge_df = mlb_history_df.merge(hof_cleaned, on='name', how='outer')
final_df = merge_df.fillna('no').rename(columns={"position": "hall_of_fame"})
final_df

,player_id,name,at_bats,runs,hits,runs_batted_in,bases_on_balls,batting_average,doubles,triples,home_runs,stolen_bases,hall_of_fame
0,whitede01,Deacon White,6624,1140,2067,988,308,0.312,270,98,24,70,3B
1,mcveyca01,Cal McVey,2513,555,869,449,30,0.346,133,44,11,26,no
2,barnero01,Ross Barnes,2391,698,860,346,116,0.36,146,47,6,103,no
3,peterjo01,John Peters,2693,373,748,249,25,0.278,92,12,3,14,no
4,peterjo02,John Peters,302,22,80,47,16,0.265,13,1,7,1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18801,no,Pedro Martinez,no,no,no,no,no,no,no,no,no,no,P
18802,no,John Schuerholz,no,no,no,no,no,no,no,no,no,no,EXEC
18803,no,Bud Selig,no,no,no,no,no,no,no,no,no,no,EXEC
18804,no,Edgar Martínez,no,no,no,no,no,no,no,no,no,no,DH


In [ ]:
players_df = final_df.sort_valu

In [13]:
# export as a csv to Resources folder
final_df.to_csv('Resources/player_data.csv', index=False)